In [1]:
import sympy as sp
from lbmpy.stencils import LBStencil, Stencil
from lbmpy.moments import MOMENT_SYMBOLS, moment_matrix

In [2]:
d3q19 = LBStencil(Stencil.D3Q19)
Q = len(d3q19)
c = sp.Matrix(d3q19)

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2+y**2+z**2
c4 = c2**2

moments = [
    one,
    x,
    y,
    z,
    c2-1,
    3*x**2-c2,
    y**2-z**2,
    x*y,
    y*z,
    z*x,
    (3*c2-5)*x,
    (3*c2-5)*y,
    (3*c2-5)*z,
    (y**2-z**2)*x,
    (z**2-x**2)*y,
    (x**2-y**2)*z,
    3*c4-6*c2+1,
    (2*c2-3)*(3*x**2-c2),
    (2*c2-3)*(y**2-z**2)
]

M = moment_matrix(moments, stencil=d3q19)

N = [ sum([w[i]*M[a,i]**2 for i in range (Q)]) for a in range(Q) ]

In [3]:
rho = sp.symbols('rho', real=True)
ux, uy, uz = sp.symbols('u_x, u_y, u_z', real=True)
u = sp.Matrix([ux,uy, uz])
pb = sp.symbols('p_b', real=True)
kappa = sp.symbols('kappa', real=True)

Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y,z] ] for a in [x,y,z] ])
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y,z] ])
G = kappa*Drho2

five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

cs = sp.sqrt(sp.Rational(1,3))

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(3) ]) \
        + sum([ ww[i][j][(j+1)%3]/cs**2*G[j,(j+1)%3] for j in range(3) ])
    return f

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])

In [4]:
rho0 = sp.symbols('rho_0', real=True)
k = sp.symbols('k', real=True)
cs = sp.symbols('c_s', cls=sp.Function, real=True)(k)
fbar = feq.subs({pb:rho*cs**2,kappa:0}).subs({rho:rho0,ux:0,uy:0,uz:0})
S = sp.symbols('S', cls=sp.Function)(k)/rho0*sp.diag(*fbar)
S = sp.symbols('k_{B}T', real=True)/cs**2*sp.diag(*fbar)
G = sp.simplify(M@S@M.T)

In [5]:
# collision operator
lb, ls, lo, le = sp.symbols('lambda_b lambda_s lambda_o lambda_e', real=True)
d = 3*(cs**2 - sp.Rational(1,3))

lambdas = [0]*4 + [lb]*1 + [ls]*5 + [lo]*6 + [le]*3
Lambda = sp.diag(*lambdas)
Lambda[4,0] = -lambdas[4]*d
Lambda[16,0] = lambdas[16]*d

In [6]:
L = sp.eye(Q)+Lambda
Xi = sp.simplify(G-L@G@L.T)
Xi

Matrix([
[0, 0, 0, 0,                                                                                                                                       0,                                      0,                                        0,                                      0,                                      0,                                      0,                                      0,                                      0,                                      0,                                        0,                                        0,                                        0,                                                                                                                                       0,                                      0,                                        0],
[0, 0, 0, 0,                                                                                                                                       0,                        

In [7]:
from sympy.printing.c import C99CodePrinter
code = C99CodePrinter().doprint(Xi.subs({cs:sp.symbols('c_s')}), assign_to=sp.MatrixSymbol('Xi', Q, Q))

In [8]:
beta, rho, rhov, rhol = sp.symbols("beta rho rho_v rho_l")
fb = beta*(rho-rhov)**2*(rho-rhol)**2
pb = sp.simplify(rho*sp.Derivative(fb,rho).doit() - fb)
cs = sp.simplify(rho*sp.Derivative(fb,rho,2).doit())

In [9]:
pb

beta*(rho - rho_l)*(rho - rho_v)*(2*rho*(2*rho - rho_l - rho_v) - (rho - rho_l)*(rho - rho_v))

In [10]:
cs

2*beta*rho*((rho - rho_l)**2 + 4*(rho - rho_l)*(rho - rho_v) + (rho - rho_v)**2)